In [1]:
! grep '>' -c raw_data/coi_complete_set_with_additions.txt

99


In [2]:
from Bio import SeqIO

file = SeqIO.parse('raw_data/coi_complete_set_with_additions.txt', 'fasta')
# print(len([rec for rec in file]))

recs = [rec for rec in file]

with open('raw_data/coi_complete_set_with_additions.fa', 'w') as out_f:
    SeqIO.write(recs, out_f, 'fasta')
    print(len(recs))

99


In [3]:
import uuid

def my_random_string(string_length=10):
    """Returns a random string of length string_length."""
    random = str(uuid.uuid4()) # Convert UUID format to a Python string.
    random = random.upper() # Make all characters uppercase.
    random = random.replace("-","") # Remove the UUID '-'.
    return random[0:string_length] # Return the random string.

# print(my_random_string(6)) # For example, D9E50C

from Bio import SeqIO

names = {}

recs_out = []

recs = SeqIO.parse('raw_data/coi_complete_set_with_additions.fa', 'fasta')
for el in recs:
    rndm = my_random_string(10)
    names[rndm] = [el.description, len(el)]
    el.name = rndm
    el.description = rndm
    el.id = rndm
    recs_out.append(el)    

# print(names)

import pandas as pd

df = pd.DataFrame.from_dict(names, orient='index', columns=['description', 'length'])
print(df)
df.to_csv('raw_data/stats_final.tsv', sep='\t')
SeqIO.write(recs_out, 'raw_data/formated_ids_final.fa', 'fasta')

                                                  description  length
D672B9D3B6                       K11-4B P.bursaria MT078144.1     769
0DA736A8B4                HSG1-11 P.chlorelligerum KX669632.1     760
C86BA12696      P.multimicronucleatum Wg8 JF741271.1  USA, VA     638
2CA228C318   P.multimicronucleatum OLI-Parame-mult-01 FJ90...     704
8A4F393EE2     P.multimicronucleatum MB2-5 JF741245.1 Moldova     638
...                                                       ...     ...
E9B50906F0                            IP P. bursaria JF304186     723
94EE43C368                        Pb1C30 P. bursaria JX082041     769
017078C790          OLI-Parame-burs-01 P. bursaria FJ905152.1     769
6EA216BAD7                         P. chlorelligerum KX669634     760
EB7EE0E3E7                     ShKm41 Kemerovo region, Russia     748

[99 rows x 2 columns]


99

In [4]:
%%time
! mafft --auto --reorder raw_data/formated_ids_final.fa > alignments/mafft_aln_fianl.fa

outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
All-to-all alignment.
tbfast-pair (nuc) Version 7.429
alg=L, model=DNA200 (2), 2.00 (6.00), -0.10 (-0.30), noshift, amax=0.0
0 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
   90 / 99
done.

Progressive alignment ... 
STEP    97 /98 
Reallocating..done. *alloclen = 2542
STEP    98 /98 
done.
tbfast (nuc) Version 7.429
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
1 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 0
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
nadd = 16
Loading 'hat3' ... done.
generating a scoring matrix for nucleotide (dist=200) ... done

   90 / 99
Segmen

In [21]:
from ete3 import Tree, faces, AttrFace, TreeStyle, NodeStyle
import pandas as pd

df = pd.read_csv('./raw_data/stats_final.tsv', sep='\t', index_col=0)
# print(df)

with open('trees/tree_3/con_50_majrule.newick', 'r') as in_f:
    with open('trees/tree_3/mb_ete_tree.newick', 'w') as out_f:
        for line in in_f:
            line = line.strip()
            for el in df.index:
                if el in line:
                    new_name = '"'+ df['description'][el]+'"'
                    line = line.replace(el, new_name)
            out_f.write(line)
            print(line)

((((((("SD11-9 Russia, Saint Petersburg":0.00175383,"V105-1 MN264579.1 (Rautian) Russia, Vologda region":0.00192297):0,"PO16-1 (COIlongF needed) Russia, Pskov region":0.00647963):0,"CB8-2 MN264581.1 (Rautian) China":0.0045835):0,("HSG3-10 Russia, Saint Petersburg":0.00923455,(("P.multimicronucleatum TRB101-1 JF741262.1 Russia, Baikal":0.00448398,"P.multimicronucleatum TRB101-4 JF741263.1 Russia, Baikal":0.00186382):0,"ShKm41 Kemerovo region, Russia":0.0024948)1.00:0.047936)0.72:0.00960004):0,"SP-1 Russia, St Petersburg":0.00937687)1.00:0.130877,(" “Paramecium germanicum” GO1 KM091240.1 Germany":0.181628,((" HSG1-11 P.chlorelligerum KX669632.1":0.00154148," P. chlorelligerum KX669634":0.00154901)1.00:0.687894,(" OLI-Parame-burs-01 P. bursaria FJ905152.1":0.0952012,(" K11-4B P.bursaria MT078144.1":0.00158917,("Pb1C30 P. bursaria JX082041":0.00347288,"IP P. bursaria JF304186":0.00340667):0)1.00:0.144219)1.00:0.531424)1.00:0.196557)0.81:0.0701521)0.82:0.0490515,((((((((((((" P.caudatum V-1

In [23]:
from ete3 import Tree, faces, AttrFace, TreeStyle, NodeStyle
import pandas as pd

df = pd.read_csv('./raw_data/stats_final.tsv', sep='\t', index_col=0)
# print(df)

with open('trees/tree_3/con_50_majrule.newick', 'r') as in_f:
    with open('trees/tree_3/tmb_tree.newick', 'w') as out_f:
        for line in in_f:
            line = line.strip()
            for el in df.index:
                if el in line:
                    new_name = df['description'][el].replace(',', '')
                    line = line.replace(el, new_name)
            out_f.write(line)
            print(line)

(((((((SD11-9 Russia Saint Petersburg:0.00175383,V105-1 MN264579.1 (Rautian) Russia Vologda region:0.00192297):0,PO16-1 (COIlongF needed) Russia Pskov region:0.00647963):0,CB8-2 MN264581.1 (Rautian) China:0.0045835):0,(HSG3-10 Russia Saint Petersburg:0.00923455,((P.multimicronucleatum TRB101-1 JF741262.1 Russia Baikal:0.00448398,P.multimicronucleatum TRB101-4 JF741263.1 Russia Baikal:0.00186382):0,ShKm41 Kemerovo region Russia:0.0024948)1.00:0.047936)0.72:0.00960004):0,SP-1 Russia St Petersburg:0.00937687)1.00:0.130877,( “Paramecium germanicum” GO1 KM091240.1 Germany:0.181628,(( HSG1-11 P.chlorelligerum KX669632.1:0.00154148, P. chlorelligerum KX669634:0.00154901)1.00:0.687894,( OLI-Parame-burs-01 P. bursaria FJ905152.1:0.0952012,( K11-4B P.bursaria MT078144.1:0.00158917,(Pb1C30 P. bursaria JX082041:0.00347288,IP P. bursaria JF304186:0.00340667):0)1.00:0.144219)1.00:0.531424)1.00:0.196557)0.81:0.0701521)0.82:0.0490515,(((((((((((( P.caudatum V-1 MT078150.1 Mexico:0.00160915,P.caudatum 

In [28]:
import pandas as pd

df = pd.read_csv('./raw_data/stats_final.tsv', sep='\t', index_col=0)
# print(df)

with open('trees/tree_3/mb_tree.newick', 'r') as in_f:
    with open('trees/tree_3/mb_ete_tree_2.newick', 'w') as out_f:
        for line in in_f:
            line = line.strip()
            for ind, name in zip(df.index, df.description):
                # print(ind, name)
                name = name.lstrip(' ')
                name = name.replace(',', '')
                if name in line:
                    new_name = '"' + df['description'][ind] + '"'
                    line = line.replace(name, new_name)
            out_f.write(line)

In [ ]:
from ete3 import Tree, faces, AttrFace, TreeStyle, NodeStyle

t = Tree('trees/tree_3/mb_ete_tree_2.newick', quoted_node_names=True)
# print(t)

for node in t.traverse():
    node.img_style['size'] = 0
    
t.ladderize(direction=1)
ts = TreeStyle()
ts.show_branch_support = True
ts.scale =  700
# ts.mode = "c"
t.show(tree_style=ts)
#t.render('trees/tree_3/mb_ete_tree.pdf', dpi=1200, tree_style=ts)